# Create Deployment

This notebook allows you to deploy BPMN and DMN files to Operaton/Camunda 7 engine.

## Features
- Select BPMN and DMN files from the current working directory
- Configure deployment name, source, and tenant ID
- Deploy selected files as a single deployment

## Usage
1. Run all cells in order
2. Select files to deploy (hold Ctrl/Cmd for multiple)
3. Configure deployment settings
4. Click "Deploy" to create the deployment

In [ ]:
# Initialize environment and imports
import operaton
from operaton import Operaton
import js
import os

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Initialize environment (loads API configuration from localStorage)
await operaton.load_env()
print("Environment loaded successfully")

In [ ]:
# Helper function to list BPMN and DMN files
def get_deployable_files():
    """List all .bpmn and .dmn files in the current working directory."""
    files = []
    try:
        for entry in os.listdir('.'):
            if entry.endswith('.bpmn') or entry.endswith('.dmn'):
                files.append(entry)
    except Exception as e:
        print(f"Error listing files: {e}")
    return sorted(files)

# Scan for files
available_files = get_deployable_files()
print(f"Found {len(available_files)} deployable files (.bpmn, .dmn)")

In [ ]:
# Create widgets for deployment configuration

# File selection
file_select = widgets.SelectMultiple(
    options=available_files,
    description='Files:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px', height='200px'),
    disabled=len(available_files) == 0
)

# Refresh files button
refresh_button = widgets.Button(
    description='Refresh Files',
    button_style='',
    icon='refresh',
    layout=widgets.Layout(width='150px')
)

# Deployment name
deployment_name = widgets.Text(
    value='',
    placeholder='Enter deployment name',
    description='Name:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# Deployment source
deployment_source = widgets.Text(
    value='jupyter-notebook',
    placeholder='Enter deployment source',
    description='Source:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# Tenant ID
tenant_id = widgets.Text(
    value='',
    placeholder='Enter tenant ID (optional)',
    description='Tenant ID:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# Enable duplicate filtering
duplicate_filtering = widgets.Checkbox(
    value=True,
    description='Enable duplicate filtering (skip if unchanged)',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# Deploy changes only
deploy_changed_only = widgets.Checkbox(
    value=False,
    description='Deploy changed resources only',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# Deploy button
deploy_button = widgets.Button(
    description='Deploy',
    button_style='primary',
    icon='upload',
    layout=widgets.Layout(width='150px')
)

# Output area
output_area = widgets.Output()

In [ ]:
# Event handlers

def on_refresh_click(button):
    """Refresh the list of deployable files."""
    global available_files
    available_files = get_deployable_files()
    file_select.options = available_files
    file_select.disabled = len(available_files) == 0
    with output_area:
        clear_output()
        print(f"Found {len(available_files)} deployable files")

def on_deploy_click(button):
    """Deploy the selected files."""
    selected_files = list(file_select.value)
    
    if not selected_files:
        with output_area:
            clear_output()
            print("⚠️ Please select at least one file to deploy")
        return
    
    name = deployment_name.value.strip()
    if not name:
        # Generate name from first selected file
        name = os.path.splitext(selected_files[0])[0]
    
    with output_area:
        clear_output()
        print(f"🚀 Deploying {len(selected_files)} file(s)...")
        print(f"   Name: {name}")
        print(f"   Source: {deployment_source.value}")
        if tenant_id.value.strip():
            print(f"   Tenant: {tenant_id.value}")
        print()
    
    try:
        # Build the multipart request
        url = os.environ["OPERATON_ENGINE_API"].rstrip("/") + "/deployment/create"
        form_data = js.FormData.new()
        
        # Add metadata
        form_data.append("deployment-name", name)
        form_data.append("deployment-source", deployment_source.value)
        
        if tenant_id.value.strip():
            form_data.append("tenant-id", tenant_id.value.strip())
        
        if duplicate_filtering.value:
            form_data.append("enable-duplicate-filtering", "true")
        
        if deploy_changed_only.value:
            form_data.append("deploy-changed-only", "true")
        
        # Add files
        for filename in selected_files:
            with open(filename, 'r') as f:
                content = f.read()
            
            # Determine MIME type
            mime_type = "application/xml"
            if filename.endswith('.dmn'):
                mime_type = "application/xml"
            
            # Create File object
            file_blob = js.File.new([content], filename, {"type": mime_type})
            form_data.append(filename, file_blob)
            
            with output_area:
                print(f"   📄 Adding: {filename}")
        
        # Send the request
        request = js.XMLHttpRequest.new()
        request.open("POST", url, False)
        request.setRequestHeader("X-XSRF-TOKEN", os.environ.get("OPERATON_CSRF_TOKEN", ""))
        request.send(form_data)
        
        with output_area:
            print()
            if request.status in [200, 204]:
                import json
                result = json.loads(request.responseText or '{}')
                print("✅ Deployment successful!")
                print()
                print(f"   Deployment ID: {result.get('id', 'N/A')}")
                print(f"   Name: {result.get('name', 'N/A')}")
                print(f"   Time: {result.get('deploymentTime', 'N/A')}")
                
                # Show deployed resources
                deployed_resources = result.get('deployedProcessDefinitions', {})
                deployed_decisions = result.get('deployedDecisionDefinitions', {})
                deployed_req = result.get('deployedDecisionRequirementsDefinitions', {})
                
                if deployed_resources:
                    print()
                    print("   Deployed Process Definitions:")
                    for key, defn in deployed_resources.items():
                        print(f"      - {defn.get('name', defn.get('key'))} (v{defn.get('version')})")
                
                if deployed_decisions:
                    print()
                    print("   Deployed Decision Definitions:")
                    for key, defn in deployed_decisions.items():
                        print(f"      - {defn.get('name', defn.get('key'))} (v{defn.get('version')})")
                
                if deployed_req:
                    print()
                    print("   Deployed Decision Requirements Definitions:")
                    for key, defn in deployed_req.items():
                        print(f"      - {defn.get('name', defn.get('key'))} (v{defn.get('version')})")
                
                # Check if nothing was deployed (duplicate filtering)
                if not deployed_resources and not deployed_decisions and not deployed_req:
                    print()
                    print("   ℹ️ No new resources deployed (duplicate filtering active)")
            else:
                print(f"❌ Deployment failed with status {request.status}")
                print(f"   {request.responseText}")
                
    except Exception as e:
        with output_area:
            print(f"❌ Error during deployment: {e}")
            import traceback
            traceback.print_exc()

# Attach event handlers
refresh_button.on_click(on_refresh_click)
deploy_button.on_click(on_deploy_click)

In [ ]:
# Display the deployment interface
display(widgets.VBox([
    widgets.HTML('<h2>📦 Create Deployment</h2>'),
    widgets.HTML('<p>Select BPMN/DMN files to deploy to the Operaton engine.</p>'),
    widgets.HBox([
        widgets.VBox([
            widgets.HTML('<strong>Select files to deploy:</strong>'),
            file_select,
            refresh_button
        ]),
        widgets.VBox([
            widgets.HTML('<strong>Deployment settings:</strong>'),
            deployment_name,
            deployment_source,
            tenant_id,
            duplicate_filtering,
            deploy_changed_only
        ], layout=widgets.Layout(margin='0 0 0 20px'))
    ]),
    widgets.HTML('<hr>'),
    deploy_button,
    output_area
]))